In [1]:
#Libraries import

import numpy as np
import pandas as pd
import sklearn
import nltk
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
# Read dataframes

df_movies = pd.read_csv('dataset/movies.csv')
df_links = pd.read_csv('dataset/links.csv')
df_ratings = pd.read_csv('dataset/ratings.csv')
df_genome_tags = pd.read_csv('dataset/genome-tags.csv')
df_genome_scores = pd.read_csv('dataset/genome-scores.csv')
df_tags = pd.read_csv('dataset/tags.csv')


In [3]:
#df_tags

In [4]:
#df_genome_scores

In [5]:
#df_genome_tags

In [6]:
#df_ratings

In [7]:
#df_movies

In [8]:
#df_links

# Data preprocessing

## Data cleaning

In [9]:
df_genome=pd.merge(df_genome_scores,df_genome_tags)
df_genome=df_genome[df_genome.relevance > 0.3]
df_genome.tail(500)

,movieId,tagId,relevance,tag
14840190,4472,1127,0.39900,zombie
14840198,4480,1127,0.36600,zombie
14840211,4493,1127,0.31600,zombie
14840215,4497,1127,0.92000,zombie
14840250,4532,1127,0.96500,zombie
...,...,...,...,...
14862143,160112,1128,0.75275,zombies
14862179,162082,1128,0.98425,zombies
14862222,164367,1128,0.97300,zombies
14862316,168498,1128,0.88225,zombies


## Data preparation

In [10]:
ps = PorterStemmer()

df_genome['stemmed']=df_genome.apply(lambda row: ps.stem(row['tag']), axis=1)
df_genome
#f = lambda x:" ".join(x for x in x.split() if x not in stop)
#df_genome_tags['stop']=df_genome_tags.apply(f)
#df_genome_tags['tokenized_sents'] = df_genome_tags.apply(lambda row: nltk.word_tokenize(row['tag']), axis=1)

,movieId,tagId,relevance,tag,stemmed
9,10,1,0.99950,007,007
366,380,1,0.58500,007,007
497,517,1,0.32275,007,007
607,648,1,0.62200,007,007
682,743,1,0.54100,007,007
...,...,...,...,...,...
14862143,160112,1128,0.75275,zombies,zombi
14862179,162082,1128,0.98425,zombies,zombi
14862222,164367,1128,0.97300,zombies,zombi
14862316,168498,1128,0.88225,zombies,zombi


In [11]:
df_genome['grpId']=df_genome.groupby('stemmed').ngroup().add(1)
df_genome['grpId'] = df_genome.grpId.astype(str)
df_genome.tail(500) 

,movieId,tagId,relevance,tag,stemmed,grpId
14840190,4472,1127,0.39900,zombie,zombi,1078
14840198,4480,1127,0.36600,zombie,zombi,1078
14840211,4493,1127,0.31600,zombie,zombi,1078
14840215,4497,1127,0.92000,zombie,zombi,1078
14840250,4532,1127,0.96500,zombie,zombi,1078
...,...,...,...,...,...,...
14862143,160112,1128,0.75275,zombies,zombi,1078
14862179,162082,1128,0.98425,zombies,zombi,1078
14862222,164367,1128,0.97300,zombies,zombi,1078
14862316,168498,1128,0.88225,zombies,zombi,1078


###### Now df_genome includes all tags with relevance higher than 0.3 which tag is stemmed and indexed with grpId

In [12]:
def _concatenate_tags_of_movie(tags):
    tags_as_str = ' '.join(set(tags))
    return tags_as_str

In [13]:
df_tags_per_movie = df_genome.groupby('movieId')['grpId'].agg(_concatenate_tags_of_movie)
df_tags_per_movie.name = 'movie_tags'
df_tags_per_movie = df_tags_per_movie.reset_index()
df_tags_per_movie


,movieId,movie_tags
0,1,723 348 403 864 917 564 449 484 431 1065 728 3...
1,2,961 145 917 564 886 1029 449 779 398 530 105 4...
2,3,426 715 470 881 29 279 874 886 699 186 834 432...
3,4,407 426 833 715 470 816 963 106 874 170 589 69...
4,5,426 833 715 332 470 503 963 361 819 886 699 18...
...,...,...
13171,185435,886 699 1055 779 704 601 885 448 288 736 773 2...
13172,185585,715 470 986 258 29 239 361 917 972 699 186 432...
13173,186587,723 385 69 986 864 917 974 819 886 1030 699 10...
13174,187593,723 348 833 308 313 403 986 864 917 543 1036 7...


In [14]:
df_tags_per_movie_text=df_genome.groupby('movieId')['stemmed'].agg(_concatenate_tags_of_movie)
df_tags_per_movie_text = df_tags_per_movie_text.reset_index()
df_tags_per_movie_text=pd.merge(df_tags_per_movie_text,df_tags_per_movie)
df_tags_per_movie_text

,movieId,stemmed,movie_tags
0,1,original plot good sequel interest oscar (best...,723 348 403 864 917 564 449 484 431 1065 728 3...
1,2,interest supernatur jungl mytholog weird runaw...,961 145 917 564 886 1029 449 779 398 530 105 4...
2,3,slapstick foul languag women good sequel nudit...,426 715 470 881 29 279 874 886 699 186 834 432...
3,4,adulteri first contact bad plot chick flick nu...,407 426 833 715 470 816 963 106 874 170 589 69...
4,5,slapstick father daughter relationship good se...,426 833 715 332 470 503 963 361 819 886 699 18...
...,...,...,...
13171,185435,slapstick sport nudity (full frontal - brief) ...,886 699 1055 779 704 601 885 448 288 736 773 2...
13172,185585,so bad it's funni bad plot women nudity (full ...,715 470 986 258 29 239 361 917 972 699 186 432...
13173,186587,foul languag sport women nudity (full frontal ...,723 385 69 986 864 917 974 819 886 1030 699 10...
13174,187593,slapstick foul languag original plot coming of...,723 348 833 308 313 403 986 864 917 543 1036 7...


In [15]:
df_avg_ratings  = df_ratings.groupby('movieId')['rating'].agg(['mean', 'median', 'size'])
df_avg_ratings.columns = ['rating_mean', 'rating_median', 'num_ratings']
df_avg_ratings = df_avg_ratings.reset_index()
#df_avg_ratings.num_ratings.hist(bins=range(0,20,5))
df_avg_ratings=df_avg_ratings[df_avg_ratings.num_ratings>5]
df_avg_ratings

,movieId,rating_mean,rating_median,num_ratings
0,1,3.886649,4.00,68469
1,2,3.246583,3.00,27143
2,3,3.173981,3.00,15585
3,4,2.874540,3.00,2989
4,5,3.077291,3.00,15474
...,...,...,...,...
53422,192697,3.363636,4.00,11
53454,192777,2.714286,3.00,7
53477,192849,3.318182,3.50,11
53592,193185,2.650000,3.75,10


In [16]:
df_movies_with_ratings = pd.merge(df_movies, df_avg_ratings, how='left', on='movieId')
df_data = pd.merge(df_movies_with_ratings, df_tags_per_movie_text, how='left', on='movieId')
df_data_with_tags = df_data[~df_data.movie_tags.isnull()].reset_index(drop=True)
df_data_with_tags

,movieId,title,genres,rating_mean,rating_median,num_ratings,stemmed,movie_tags
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.886649,4.0,68469.0,original plot good sequel interest oscar (best...,723 348 403 864 917 564 449 484 431 1065 728 3...
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.246583,3.0,27143.0,interest supernatur jungl mytholog weird runaw...,961 145 917 564 886 1029 449 779 398 530 105 4...
2,3,Grumpier Old Men (1995),Comedy|Romance,3.173981,3.0,15585.0,slapstick foul languag women good sequel nudit...,426 715 470 881 29 279 874 886 699 186 834 432...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.874540,3.0,2989.0,adulteri first contact bad plot chick flick nu...,407 426 833 715 470 816 963 106 874 170 589 69...
4,5,Father of the Bride Part II (1995),Comedy,3.077291,3.0,15474.0,slapstick father daughter relationship good se...,426 833 715 332 470 503 963 361 819 886 699 18...
...,...,...,...,...,...,...,...,...
13171,185435,"Game Over, Man! (2018)",Action|Comedy,2.562500,3.0,64.0,slapstick sport nudity (full frontal - brief) ...,886 699 1055 779 704 601 885 448 288 736 773 2...
13172,185585,Pacific Rim: Uprising (2018),Action|Fantasy|Sci-Fi,2.830128,3.0,312.0,so bad it's funni bad plot women nudity (full ...,715 470 986 258 29 239 361 917 972 699 186 432...
13173,186587,Rampage (2018),Action|Adventure|Sci-Fi,2.945755,3.0,212.0,foul languag sport women nudity (full frontal ...,723 385 69 986 864 917 974 819 886 1030 699 10...
13174,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi,3.838947,4.0,1633.0,slapstick foul languag original plot coming of...,723 348 833 308 313 403 986 864 917 543 1036 7...


# Data enconding

In [17]:
tf_idf = TfidfVectorizer()
sparse_matrix = tf_idf.fit_transform(df_data_with_tags.stemmed)
#m2m = cosine_similarity(sparse_matrix)
df_tfidf_similar = pd.DataFrame(cosine_similarity(sparse_matrix))
index_to_movie_id = df_data_with_tags['movieId']
df_tfidf_similar.columns = [str(index_to_movie_id[int(col)]) for col in df_tfidf_similar.columns]
df_tfidf_similar.index = [index_to_movie_id[idx] for idx in df_tfidf_similar.index]
df_tfidf_similar.head()

,1,2,3,4,5,6,7,8,9,10,...,184987,184997,185029,185135,185425,185435,185585,186587,187593,187595
1,1.000000,0.462745,0.215649,0.220860,0.228145,0.410830,0.296765,0.269200,0.076516,0.241485,...,0.266186,0.391125,0.580089,0.543576,0.180293,0.177065,0.227055,0.304636,0.493132,0.459971
2,0.462745,1.000000,0.183530,0.235638,0.171096,0.143772,0.288950,0.290667,0.278500,0.288449,...,0.400362,0.298788,0.322306,0.412553,0.217253,0.200669,0.227634,0.368936,0.333468,0.424950
3,0.215649,0.183530,1.000000,0.263559,0.503608,0.167767,0.357936,0.154670,0.070063,0.191633,...,0.159959,0.268142,0.252743,0.219720,0.137448,0.193388,0.249527,0.240640,0.272959,0.300338
4,0.220860,0.235638,0.263559,1.000000,0.287089,0.124760,0.401416,0.356245,0.077829,0.099731,...,0.248478,0.296691,0.292587,0.269218,0.286829,0.227345,0.204242,0.283440,0.271837,0.326868
5,0.228145,0.171096,0.503608,0.287089,1.000000,0.126927,0.407728,0.175734,0.083730,0.179651,...,0.180823,0.325834,0.217449,0.223583,0.164896,0.178076,0.188214,0.215157,0.251907,0.271792


## User profile

In [18]:
n_user=input()
df_user_ratings = df_ratings[df_ratings.userId == int(n_user)]
df_user_data_with_tags = df_data_with_tags.reset_index().merge(df_user_ratings, on='movieId')

5


In [19]:
df_user_data_with_tags.movie_tags

0     723 348 308 230 145 864 628 1036 446 753 699 4...
1     723 348 308 313 230 864 628 446 753 699 443 10...
2     723 348 308 313 385 230 986 1008 864 1036 446 ...
3     723 348 308 313 230 986 1008 479 628 543 446 7...
4     723 348 833 308 230 145 864 917 628 446 753 69...
                            ...                        
67    723 348 193 308 313 230 145 864 628 1036 446 6...
68    723 348 308 313 230 1008 864 479 443 449 166 1...
69    723 348 308 313 385 230 864 628 1036 446 753 6...
70    723 348 308 543 974 699 571 449 484 166 1065 5...
71    723 348 308 230 310 628 1017 446 753 699 443 4...
Name: movie_tags, Length: 72, dtype: object

In [20]:
#standardize ratings
df_user_data_with_tags['weight'] = df_user_data_with_tags['rating']/5.

Weighted mean of the user’s ratings and the TF-IDF vector representations of the respective movies

In [21]:
user_profile = np.dot(sparse_matrix[df_user_data_with_tags['index'].values].toarray().T, df_user_data_with_tags['weight'].values)
df_user_data_with_tags[df_user_data_with_tags.rating>=4]

,index,movieId,title,genres,rating_mean,rating_median,num_ratings,stemmed,movie_tags,userId,rating,timestamp,weight
0,46,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.071687,4.0,55240.0,foul languag crazi original plot assassin nudi...,723 348 308 230 145 864 628 1036 446 753 699 4...,5,4.0,1209126049,0.8
1,49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.291959,4.5,62180.0,foul languag crazi original plot assassin nudi...,723 348 308 313 230 864 628 446 753 699 443 10...,5,5.0,1209039937,1.0
3,283,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.173971,4.5,92406.0,foul languag crazi original plot assassin uniq...,723 348 308 313 230 986 1008 479 628 543 446 7...,5,5.0,1209040010,1.0
4,305,318,"Shawshank Redemption, The (1994)",Crime|Drama,4.424188,4.5,97999.0,foul languag original plot coming of ag photog...,723 348 833 308 230 145 864 917 628 446 753 69...,5,5.0,1209040104,1.0
5,507,527,Schindler's List (1993),Drama|War,4.257502,4.5,71516.0,disabl foul languag original plot coming of ag...,723 348 560 308 833 385 1049 230 145 864 378 9...,5,4.5,1209126022,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,9029,55290,Gone Baby Gone (2007),Crime|Drama|Mystery,3.800871,4.0,4590.0,foul languag original plot coming of ag assass...,723 348 193 308 313 230 145 864 628 1036 446 6...,5,4.0,1209039858,0.8
68,9055,55765,American Gangster (2007),Crime|Drama|Thriller,3.800331,4.0,7853.0,foul languag original plot assassin interest o...,723 348 308 313 230 1008 864 479 443 449 166 1...,5,4.0,1209040037,0.8
69,9059,55820,No Country for Old Men (2007),Crime|Drama,4.006856,4.0,18523.0,foul languag original plot photographi assassi...,723 348 308 313 385 230 864 628 1036 446 753 6...,5,4.5,1209039831,0.9
70,9097,56367,Juno (2007),Comedy|Drama|Romance,3.749987,4.0,19131.0,foul languag coming of ag women nudity (full f...,723 348 308 543 974 699 571 449 484 166 1065 5...,5,5.0,1209039862,1.0


# Results

cosine similarity between the user profile vectors and content vectors to find their similarity

In [22]:
similarities = cosine_similarity(np.atleast_2d(user_profile), sparse_matrix)
rec = np.argsort(similarities)[:, ::-1]
recommendations = [i for i in rec[0] if i not in df_user_data_with_tags['index'].values]

In [23]:
summary=df_data_with_tags['title'][recommendations].head(10)
extended=pd.merge(summary,df_data_with_tags)
extended

,title,movieId,genres,rating_mean,rating_median,num_ratings,stemmed,movie_tags
0,"Three Billboards Outside Ebbing, Missouri (2017)",177593,Crime|Drama,4.009094,4.0,2749.0,foul languag original plot coming of ag photog...,723 348 308 385 230 864 1036 753 699 443 449 1...
1,Blade Runner 2049 (2017),176371,Sci-Fi,3.888662,4.0,4154.0,foul languag original plot photographi good se...,723 348 308 385 403 230 986 864 310 628 1036 4...
2,Whiplash (2014),112552,Drama,4.127808,4.5,10907.0,foul languag original plot coming of ag classi...,723 348 308 230 864 628 446 753 699 443 449 87...
3,Voices from the List (2004),86504,Documentary,4.124444,4.5,1800.0,foul languag original plot coming of ag assass...,723 348 833 308 385 1049 230 145 864 753 699 1...
4,Unforgiven (1992),1266,Drama|Western,3.985085,4.0,16627.0,foul languag original plot photographi assassi...,723 348 308 385 230 864 1036 446 564 753 699 4...
5,Amores Perros (Love's a Bitch) (2000),4235,Drama|Thriller,3.997029,4.0,6900.0,original plot photographi assassin interest os...,723 348 308 313 385 230 864 446 753 443 449 87...
6,The Godfather Trilogy: 1972-1990 (1992),172591,(no genres listed),4.339667,4.5,421.0,original plot photographi assassin nudity (ful...,723 348 308 385 230 145 864 310 726 446 753 69...
7,Mystic River (2003),6870,Crime|Drama|Mystery,3.827229,4.0,12259.0,foul languag original plot assassin nudity (fu...,723 348 193 308 313 230 145 864 628 1036 753 6...
8,Isle of Dogs (2018),183897,Animation|Comedy,3.930587,4.0,886.0,original plot coming of ag good sequel nudity ...,723 833 308 313 385 230 986 145 864 310 1036 7...
9,Whiplash (2013),166024,(no genres listed),4.202862,4.5,594.0,foul languag original plot coming of ag assass...,723 348 833 308 313 230 145 864 628 1036 699 4...
